In [2]:
#@title Librerias usadas
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.naive_bayes import GaussianNB
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import *
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import joblib
import tensorflow as tf
from tensorflow import keras
# Download the NLTK stop words if they haven't been downloaded already
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
#@title Montando Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title Importando dataset
Suicide = pd.read_csv('/content/drive/MyDrive/IA I Proyecto final/Suicide_Detection.csv')

In [ ]:
#@title Eliminando columna Unnamed y agregando la longitud de las frases como característica
Suicide = Suicide.drop('Unnamed: 0', axis=1);
Suicide['Length'] = Suicide['text'].apply(lambda x: len(x.split(' ')))
Suicide['class'] = Suicide['class'].map({'suicide': 1, 'non-suicide': 0})

In [ ]:
#@title Separando características y etiquetas
X = Suicide['text']
y = Suicide['class']

In [25]:
#@title Preprocesamiento de las frases
def preprocess_words(data):
  # Convierte todo el texto a minúscula
  data = [text.lower() for text in data]

  # Remueve signos de puntuación, y crea una matriz separando cada palabra de las frases
  tokenizer = RegexpTokenizer(r"\w+")
  data = [tokenizer.tokenize(text) for text in data]

  # Eliminando palabras más usadas del lenguaje
  stop_words = set(stopwords.words("english"))
  data = [[word for word in text if word not in stop_words] for text in data]

  # Conviertiendo las palabras a su forma básica
  from nltk.stem import SnowballStemmer
  stemmer = SnowballStemmer("english")
  data = [[stemmer.stem(word) for word in text] for text in data]

  # Junta cada palabra para volver a armar las frases ya procesadas
  data = [" ".join(text) for text in data]
  return data

def train_vectorizer(data):
  vectorizer = CountVectorizer(min_df=80)

  # Entrena un contador de vectores que actua como un kmeans para agrupar y contar palabras
  vectorizer.fit(data)

  return vectorizer

In [ ]:
#@title Preprocesando todas las frases del dataset
X = preprocess_words(X)

In [ ]:
#@title Guardando frases preprocesadas en archivos
with open('/content/drive/MyDrive/IA I Proyecto final/processed_data', 'wb') as fp:
    pickle.dump(X, fp)

In [ ]:
#@title Guardando etiquetas en archivos
with open('/content/drive/MyDrive/IA I Proyecto final/processed_data_y', 'wb') as fp:
    pickle.dump(y, fp)

In [4]:
#@title Recuperando características desde los archivos
with open ('/content/drive/MyDrive/IA I Proyecto final/processed_data', 'rb') as fp:
    X = pickle.load(fp)

In [5]:
#@title Recuperando etiquetas desde los archivos
with open ('/content/drive/MyDrive/IA I Proyecto final/processed_data_y', 'rb') as fp:
    y = pickle.load(fp)

In [29]:
#@title Obteniendo vectorizer con las palabras contadas (bag of words)
vec = train_vectorizer(X)

In [30]:
#@title Guardando vectorizer en archivos
joblib.dump(vec, '/content/drive/MyDrive/IA I Proyecto final/vectorizer.joblib')

['/content/drive/MyDrive/IA I Proyecto final/vectorizer.joblib']

In [6]:
#@title Obteniendo vectorizer desde archivos
vec = joblib.load('/content/drive/MyDrive/IA I Proyecto final/vectorizer.joblib')

In [7]:
#@title Obteniendo datos transformados con el vectorizer (palabras separadas y su frecuencia)
X = vec.transform(X)

In [32]:
X.shape

(232074, 5663)

In [8]:
#@title Separando datos de entrenamiento
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.6, random_state=21)
del X
del y

In [ ]:
#@title Entrenando Gaussian Naive Bayes
GNB = GaussianNB()
GNB.fit(X_train.toarray(), y_train)

GaussianNB()

In [ ]:
#@title Accuracy GNB
print("%.3f"%accuracy_score(GNB.predict(X_test[:2000].toarray()), y_test[:2000]))

0.789


In [ ]:
#@title Entrenando DecisionTree
DT = DecisionTreeClassifier(max_depth=10)
DT.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=10)

In [ ]:
#@title Accuracy DT
print(accuracy_score(DT.predict(X_test[:2000]), y_test[:2000]))

0.864


In [ ]:
#@title Entrenando Random Forest
RF = RandomForestClassifier()
RF.fit(X_train,y_train)

RandomForestClassifier()

In [ ]:
#@title Accuracy RF
print(accuracy_score(RF.predict(X_test[:2000]), y_test[:2000]))

0.9


In [ ]:
#@title Entrenando Support Vector Machine
SV = SVC()
SV.fit(X_train,y_train)

SVC()

In [ ]:
#@title Accuracy SVM
print(accuracy_score(SV.predict(X_test[:2000]), y_test[:2000]))

0.915


In [16]:
#@title Entrenando Red Neuronal
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256 ,input_shape=(X_train.shape[1],), activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])

model.compile(optimizer=tf.keras.optimizers.SGD(),  #AdamOptimizer(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train.toarray()[:10000], y_train[:10000], epochs=8)

Epoch 1/8
313/313 [==============================] - 2s 3ms/step - loss: 0.5218 - accuracy: 0.8039
Epoch 2/8
313/313 [==============================] - 1s 4ms/step - loss: 0.3498 - accuracy: 0.8697
Epoch 3/8
313/313 [==============================] - 1s 5ms/step - loss: 0.2786 - accuracy: 0.8977
Epoch 4/8
313/313 [==============================] - 1s 5ms/step - loss: 0.2375 - accuracy: 0.9140
Epoch 5/8
313/313 [==============================] - 1s 4ms/step - loss: 0.2097 - accuracy: 0.9249
Epoch 6/8
313/313 [==============================] - 1s 3ms/step - loss: 0.1871 - accuracy: 0.9344
Epoch 7/8
313/313 [==============================] - 1s 4ms/step - loss: 0.1673 - accuracy: 0.9417
Epoch 8/8
313/313 [==============================] - 1s 3ms/step - loss: 0.1505 - accuracy: 0.9469


In [17]:
#@title Evaluating the model
test_loss, test_acc = model.evaluate(X_test.toarray()[:2000], y_test[:2000])

print('Test accuracy:', test_acc)

63/63 [==============================] - 0s 4ms/step - loss: 0.2288 - accuracy: 0.9240
Test accuracy: 0.9240000247955322


In [ ]:
#@title Guardando clasificadores entrenados en archivos
dt_file = '/content/drive/MyDrive/IA I Proyecto final/decision_tree_trained.joblib'
gnb_file = '/content/drive/MyDrive/IA I Proyecto final/gnb_trained.joblib'
rf_file = '/content/drive/MyDrive/IA I Proyecto final/rf_trained.joblib'
svc_file = '/content/drive/MyDrive/IA I Proyecto final/svc_trained.joblib'
nn_file = '/content/drive/MyDrive/IA I Proyecto final/neural_network_trained.joblib'

joblib.dump(DT, dt_file)
joblib.dump(GNB, gnb_file)
joblib.dump(RF, rf_file)
joblib.dump(SV, svc_file)
joblib.dump(model, nn_file)

['svc_trained.joblib']

In [8]:
#@title Recuperando clasificadores entrenados desde los archivos
DT = joblib.load('/content/drive/MyDrive/IA I Proyecto final/decision_tree_trained.joblib')
GNB = joblib.load('/content/drive/MyDrive/IA I Proyecto final/gnb_trained.joblib')
RF = joblib.load('/content/drive/MyDrive/IA I Proyecto final/rf_trained.joblib')
SV = joblib.load('/content/drive/MyDrive/IA I Proyecto final/svc_trained.joblib')
model = joblib.load('/content/drive/MyDrive/IA I Proyecto final/neural_network_trained.joblib')

# Overfitting and Underfitting

In [34]:
#@title Gaussian Naive Bayes
print("%.3f"%accuracy_score(GNB.predict(X_train[:2000].toarray()), y_train[:2000]))
print("%.3f"%accuracy_score(GNB.predict(X_test[:2000].toarray()), y_test[:2000]))

0.790
0.789


In [35]:
#@title Decision Tree
print("%.3f"%accuracy_score(DT.predict(X_train[:2000].toarray()), y_train[:2000]))
print("%.3f"%accuracy_score(DT.predict(X_test[:2000].toarray()), y_test[:2000]))

0.863
0.864


In [36]:
#@title Random Forest
print("%.3f"%accuracy_score(RF.predict(X_train[:2000].toarray()), y_train[:2000]))
print("%.3f"%accuracy_score(RF.predict(X_test[:2000].toarray()), y_test[:2000]))

0.999
0.900


In [37]:
#@title Support Vector Machine
print("%.3f"%accuracy_score(SV.predict(X_train[:2000].toarray()), y_train[:2000]))
print("%.3f"%accuracy_score(SV.predict(X_test[:2000].toarray()), y_test[:2000]))

0.909
0.915


In [19]:
#@title Neural Networks
test_loss1, test_acc1 = model.evaluate(X_train.toarray()[:2000], y_train[:2000])
test_loss2, test_acc2 = model.evaluate(X_test.toarray()[:2000], y_test[:2000])
print('%.3f'%test_acc1)
print('%.3f'%test_acc2)

63/63 [==============================] - 0s 3ms/step - loss: 0.2288 - accuracy: 0.9240
0.957
0.924
